# Import Packages

In [1]:
import sys
sys.path.append('..')
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import itertools
sns.set(style="ticks")
%config IPCompleter.greedy = True
from sklearn.preprocessing import LabelEncoder

import utils

# Full Dataset

## Import Data

In [2]:
  df_calendar, df_sales, df_prices= utils.import_m5_data(reduce_memory=False)

## Transform Dataframe

In [3]:
df_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [4]:
# Add placeholders as timeframe is different for df_sales
for i in range(1914, 1970):
    df_sales[f'd_{i}'] = 'NA'

In [5]:
# transform Dataframe
df = utils.transform_dataframe(df_sales, df_calendar, df_prices)

In [6]:
# Add column with type of data (training, validation & evaluation)

# 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913 ---> training
# 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public) ---> validation
# 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private) ---> evaluation


conditions = [
    (df['date'] <= '2016-04-24'),
    (df['date'] >= '2016-04-25') & (df['date'] <= '2016-05-22'),
    (df['date'] >= '2016-05-23')]
choices = ['train', 'validation', 'evaluation']
df['data_type'] = np.select(conditions, choices)



In [7]:
# reduce memory usage
df = utils.reduce_mem_usage(df)

Mem. usage of decreased to 8072.77 Mb (26.6% reduction)


In [8]:
df.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sale,date,wm_yr_wk,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,data_type
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,train
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,train
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,2011,NaN,NaN,NaN,NaN,0,0,0,NaN,train


## Save DataFrame as CSV

In [9]:
df.to_csv(f'{utils.get_m5_root_dir()}/data/preprocessed/preprocessed_input_data.csv', index=False)

# Create Small Subset

Load full dataset if needed

In [10]:
# df = pd.read_csv(f'{utils.get_m5_root_dir()}/data/preprocessed/preprocessed_input_data.csv')

Create Subset

In [11]:
# Subset including 1 year training date and validation/evaluation data of store Texas 2, category Hobbies 
store = 'TX_2'
cat = 'HOBBIES'


df_subset = df.loc[df['store_id'] == store].loc[
    df['cat_id'] == cat].loc[df['date'] >= '2015-04-24'].loc[df['date'] <= '2016-05-22'] 

    
df_subset.reset_index(drop=True)
df_subset = utils.reduce_mem_usage(df_subset)
df_subset.to_csv(f'{utils.get_m5_root_dir()}/data/preprocessed/tx2_hobbies_1year.csv', index=False)
df_subset


Mem. usage of decreased to 30.01 Mb (0.0% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sale,date,wm_yr_wk,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,data_type
47152785,HOBBIES_1_001_TX_2_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_2,TX,d_1547,0,2015-04-24,11512,...,2015,NaN,NaN,NaN,NaN,0,0,0,8.26,train
47152786,HOBBIES_1_002_TX_2_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,TX_2,TX,d_1547,1,2015-04-24,11512,...,2015,NaN,NaN,NaN,NaN,0,0,0,3.97,train
47152787,HOBBIES_1_003_TX_2_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,TX_2,TX,d_1547,0,2015-04-24,11512,...,2015,NaN,NaN,NaN,NaN,0,0,0,2.97,train
47152788,HOBBIES_1_004_TX_2_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_2,TX,d_1547,0,2015-04-24,11512,...,2015,NaN,NaN,NaN,NaN,0,0,0,4.64,train
47152789,HOBBIES_1_005_TX_2_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,TX_2,TX,d_1547,0,2015-04-24,11512,...,2015,NaN,NaN,NaN,NaN,0,0,0,2.73,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59166405,HOBBIES_2_145_TX_2_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,TX_2,TX,d_1941,NA,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,2.88,validation
59166406,HOBBIES_2_146_TX_2_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,TX_2,TX,d_1941,NA,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,1.97,validation
59166407,HOBBIES_2_147_TX_2_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,TX_2,TX,d_1941,NA,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,0.97,validation
59166408,HOBBIES_2_148_TX_2_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,TX_2,TX,d_1941,NA,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,0.88,validation
